<a href="https://colab.research.google.com/github/leodjn/Pos_IESB/blob/main/Leonardo_Della_TRABALHO_PANDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise - DIESEL

Diesel é um óleo derivado da destilação do petróleo bruto usado como combustível nos motores a diesel/gasóleo, constituído basicamente por hidrocarbonetos. O óleo diesel é um composto formado principalmente por átomos de carbono, hidrogênio e em baixas concentrações por enxofre, nitrogênio e oxigênio. O diesel é selecionado de acordo com suas características de ignição e de escoamento, adequadas ao funcionamento dos motores ciclo diesel. É um produto pouco inflamável, medianamente tóxico, pouco volátil, límpido, isento de material em suspensão. 

![](https://lh3.googleusercontent.com/proxy/GIcndAIqPJ49RV3SUseSaPZiIpCB99AL-YFfdsNX8jBfxXj5kkwo8nIUZyxc5dx46xIVYdv7wltSblQQRHgOAH_rbstukwpUvJk-FtFHfFXajR8FVJJgCfup1qGKgFnEFgv7_4OdJaPyTqVAbm1R7LCzzOA)

No Brasil há predominância do transporte rodoviário, tanto de passageiros quanto de carga. Por esta razão, o óleo diesel é o derivado de petróleo mais consumido no país. Qualquer crescimento econômico demanda uma alta na produção e venda de Diesel. A Agência Nacional do Petróleo, Gás Natural e Biocombustíveis (ANP) autoriza a utilização do óleo diesel para outras aplicações, denominado off road, e prevê sua futura regulamentação:


>*Art. 16. O uso de óleo diesel para fins ferroviários, agropecuários, industrial e geração de energia elétrica está autorizado até que se estabeleça especificação para uso não rodoviário (off road).*

Entender correntamente o comportamento de uso do Diesel no Brasil é necessário para prever sua demanda e abastecer o crescimento do país. A empresa XPTO é uma operadora de venda de diesel no Brasil, com mais de 300 postos de combustíveis pelo país ela entende que para criação de estratégias de sucesso são necessários dados. Por isso, ela contratou um cientista de dados (você) para dar insights sobre a demanda nacional de Diesel no Brasil.

Para montar um entregável para o seu chefe será necessário responder as seguintes questões:

1. Qual é o tipo de combustível diesel mais vendido a cada ano?
2. Levando em consideração somente o ÓLEO DIESEL S-10, qual o estado da região norte teve a maior média anual de compra para o ano de 2015?
3. Considerando somente o ÓLEO DIESEL S-500, encontre os outlier nas vendas de Março. Utilize o [método IQR](https://online.stat.psu.edu/stat200/lesson/3/3.2).
4. Cálcule o total de vendas (independente do tipo de diesel) para cada ano e encontre o maior aumento percentual.
5. Selecione um tipo de combustível, um ano e um Estado a sua escolha, demonstre a diferença percentual de venda para cada trimestre do ano.
6. Colete dados populacionais e demonstre se existe uma correlação entre a medição populacional anual e a venda total de diesel S-10 anual.(Dica: [o IBGE é uma boa fonte](https://www.ibge.gov.br/apps/populacao/projecao/index.html))
7. É seguro dizer que quanto maior a população de um estado mais diesel será vendido? Justifique sua resposta com dados.
8. Qual foi o custo mensal de diesel S-10 no DF para o ano de 2020.Considere a média dos preços coletados e que o dataset de venda apresenta dados em metros cúbicos (e o de preço em R\$/litro).
9. Qual estado em 2020 gastou mais com diesel S-10 em relação a população? (um estado com 10 pessoas e que gasta 1000 reais com diesel em 2020 tem uma relação de R$ 100 reais gastos com diesel por pessoa).


## Datasets

dataset de venda de diesel: [link](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/vdpb/vct/vendas-oleo-diesel-tipo-m3-2013-2020.csv)


datasets de preço de diesel (considere a coluna 'Valor de Venda' e os datasets com o nome parecido com esse "Oleo Diesel S-500 e S-10 + GNV - \<MES>/2020"): [link](https://dados.gov.br/dataset/serie-historica-de-precos-de-combustiveis-por-revenda)

Resolução

QUESTÃO 1 - Qual é o tipo de combustível diesel mais vendido a cada ano?

In [1]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(["REGIÃO", 'ESTADO', 'UNIDADE', 'Jan','Fev',	'Mar',	'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
dfg = dfc.groupby(by=['COMBUSTÍVEL','ANO']).agg(TOTAL_VENDAS=('TOTAL','sum')).reset_index()
dfb = dfg.groupby(by=['ANO']).apply(lambda z : z.sort_values(['TOTAL_VENDAS'],ascending=[False]).head(1))
print(dfb.rename(columns={'COMBUSTÍVEL':'TIPO DE DIESEL MAIS VENDIDO', 'TOTAL_VENDAS':'TOTAL DE VENDAS NO ANO'}))

        TIPO DE DIESEL MAIS VENDIDO   ANO  TOTAL DE VENDAS NO ANO
ANO                                                              
2013 32          ÓLEO DIESEL S-500   2013             24627904.35
2014 33          ÓLEO DIESEL S-500   2014             43130158.04
2015 34          ÓLEO DIESEL S-500   2015             38813349.86
2016 35          ÓLEO DIESEL S-500   2016             35140066.91
2017 36          ÓLEO DIESEL S-500   2017             33968705.44
2018 37          ÓLEO DIESEL S-500   2018             32860853.92
2019 38          ÓLEO DIESEL S-500   2019             29412866.44
2020 23           ÓLEO DIESEL S-10   2020             30183162.04


QUESTÃO 2 - Levando em consideração somente o ÓLEO DIESEL S-10, qual o estado da região norte teve a maior média anual de compra para o ano de 2015?

In [2]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(['UNIDADE','TOTAL'], axis=1)

filtro1_reg = dfc[dfc.REGIÃO.eq('REGIÃO NORTE')].reset_index()
filtro1_ano = filtro1_reg[filtro1_reg.ANO.eq(2015)].reset_index()
filtro1_com = filtro1_ano[filtro1_ano.COMBUSTÍVEL.eq('ÓLEO DIESEL S-10 ')]

filtro1_com['MEDIA'] = filtro1_com.mean(axis=1).copy()

filtro1_com2 = filtro1_com.drop(["REGIÃO", 'Jan','Fev',	'Mar',	'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
print(filtro1_com2.sort_values(['MEDIA'],ascending=[False]).head(1).rename(columns={'MEDIA':'MAIOR MEDIA ANUAL'}).reset_index)

<bound method DataFrame.reset_index of    level_0  index        COMBUSTÍVEL   ANO ESTADO  MAIOR MEDIA ANUAL
4       74    274  ÓLEO DIESEL S-10   2015   PARÁ           54829.03>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


QUESTÃO 3 - Considerando somente o ÓLEO DIESEL S-500, encontre os outlier nas vendas de Março. Utilize o método IQR.

In [3]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(['UNIDADE','TOTAL','Jan','Fev',	'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
dfd = dfc[dfc.COMBUSTÍVEL.eq('ÓLEO DIESEL S-500 ')]

q1 = dfd['Mar'].quantile(0.25)                 
q3 = dfd['Mar'].quantile(0.75)
iqr = q3 - q1
dff = dfd.query('(@q1 - 1.5 * @iqr) <= Mar <= (@q3 + 1.5 * @iqr)')
print(dff.sort_values(['Mar'],ascending=[False]).rename(columns={'Mar':'OUTLIERS DE MARÇO'}))

            COMBUSTÍVEL   ANO  ...             ESTADO OUTLIERS DE MARÇO
988  ÓLEO DIESEL S-500   2020  ...       MINAS GERAIS         296150.56
992  ÓLEO DIESEL S-500   2020  ...             PARANÁ         292131.28
857  ÓLEO DIESEL S-500   2019  ...             PARANÁ         291240.30
184  ÓLEO DIESEL S-500   2014  ...  RIO GRANDE DO SUL         281121.92
454  ÓLEO DIESEL S-500   2016  ...  RIO GRANDE DO SUL         260186.42
..                  ...   ...  ...                ...               ...
29   ÓLEO DIESEL S-500   2013  ...           AMAZONAS            151.00
32   ÓLEO DIESEL S-500   2013  ...              AMAPÁ              0.00
28   ÓLEO DIESEL S-500   2013  ...               ACRE              0.00
30   ÓLEO DIESEL S-500   2013  ...            RORAIMA              0.00
53   ÓLEO DIESEL S-500   2013  ...   DISTRITO FEDERAL              0.00

[194 rows x 5 columns]


QUESTÂO 4 - Calcule o total de vendas (independente do tipo de diesel) para cada ano e encontre o maior aumento percentual.

In [4]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(['UNIDADE', 'ESTADO', 'REGIÃO', 'COMBUSTÍVEL','Jan','Fev',	'Mar', 'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)

dfg = dfc.groupby(by=['ANO']).agg(TOTAL_VENDAS=('TOTAL','sum')).reset_index()
dfor = dfg.sort_values(['ANO'],ascending=[False])
dfdif = dfor.set_index('ANO').pct_change(axis=0);

dffim = dfdif.sort_values(['TOTAL_VENDAS'],ascending=[False]).head(1).rename(columns={'TOTAL_VENDAS':'AUMENTO PERCENTUAL'})
print('MAIOR AUMENTO PERCENTUAL EM RELAÇÃO AO ANO ANTERIOR:')
print(dffim);

MAIOR AUMENTO PERCENTUAL EM RELAÇÃO AO ANO ANTERIOR:
      AUMENTO PERCENTUAL
ANO                     
2015                0.05


QUESTÃO 5 - Selecione um tipo de combustível, um ano e um Estado a sua escolha, demonstre a diferença percentual de venda para cada trimestre do ano.


In [5]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(['UNIDADE', 'REGIÃO' ], axis=1)

dfc_com = dfc[dfc.COMBUSTÍVEL.eq('ÓLEO DIESEL S-10 ')].copy()
dfc_ano = dfc_com[dfc_com.ANO.eq(2019)].copy()
dfc_est = dfc_ano[dfc_ano.ESTADO.eq('SÃO PAULO')].copy()

trime={'1° Trimestre':['Jan','Fev','Mar'],'2° Trimestre':['Abr','Mai','Jun'],'3° Trimestre':['Jul','Ago','Set'],'4° Trimestre':['Out','Nov','Dez']}

for trimestre in trime:
  dfc_est[trimestre]=dfc_est[trime[trimestre][0]]+dfc_est[trime[trimestre][1]]+dfc_est[trime[trimestre][2]]

dfc_est=pd.melt(dfc_est,id_vars=['COMBUSTÍVEL','ANO','ESTADO'],value_vars=[x for x in trime.keys()],var_name='TRIMESTRE', value_name='VENDAS PERÍODO').sort_values(by=['TRIMESTRE'])

dfc_est['DIFERENÇA PERCENTUAL']=dfc_est['VENDAS PERÍODO'].pct_change()

print("A diferença percentual de venda para cada trimestre do ano de 2019 do item ÓLEO DIESEL S-10 em SÂO PAULO foi:")

print(dfc_est)

A diferença percentual de venda para cada trimestre do ano de 2019 do item ÓLEO DIESEL S-10 em SÂO PAULO foi:
         COMBUSTÍVEL   ANO  ... VENDAS PERÍODO DIFERENÇA PERCENTUAL
0  ÓLEO DIESEL S-10   2019  ...     1309314.62                  nan
1  ÓLEO DIESEL S-10   2019  ...     1499814.22                 0.15
2  ÓLEO DIESEL S-10   2019  ...     1631752.64                 0.09
3  ÓLEO DIESEL S-10   2019  ...     1604095.09                -0.02

[4 rows x 6 columns]


QUESTÃO 6 - Colete dados populacionais e demonstre se existe uma correlação entre a medição populacional anual e a venda total de diesel S-10 anual.(Dica: o IBGE é uma boa fonte)


In [6]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)

dfc = df.drop(['UNIDADE', 'ESTADO', 'REGIÃO', 'Jan','Fev',	'Mar', 'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
dff = dfc[dfc.COMBUSTÍVEL.eq('ÓLEO DIESEL S-10 ')].copy()
dfg = dff.groupby(by=['ANO']).agg(TOTAL_VENDAS=('TOTAL','mean')).reset_index()

pop=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/populacao_ibge.zip', compression='zip',sep=';').rename(columns={'ano':'ANO'})
pop['ANO'] = pop['ANO'].astype(object)

pop = pop.groupby(by=['ANO','cod_municipio']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','mean')).reset_index()
pop = pop.groupby(by=['ANO']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','sum')).reset_index()

popm = pop.query('ANO >= 2013 and ANO <= 2020')
merge = popm.merge(dfg, left_on='ANO', right_on='ANO')

corre = merge['NU_POPULACAO_ESTIMADA'].corr(merge['TOTAL_VENDAS'])
print('O quoeficiente de correlação é de ',corre, '. O que demonstra uma correlação forte entre a venda media de diesel S-10 e a média da população em cada ano.')

O quoeficiente de correlação é de  0.9186642852170817 . O que demonstra uma correlação forte entre a venda media de diesel S-10 e a média da população em cada ano.


QUESTÃO 7 - É seguro dizer que quanto maior a população de um estado mais diesel será vendido? Justifique sua resposta com dados.


In [7]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)
dfc = df.drop(['UNIDADE', 'REGIÃO', 'Jan','Fev',	'Mar', 'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
dfg = dfc.groupby(by=['ANO','ESTADO']).agg(TOTAL_VENDAS=('TOTAL','mean')).reset_index()

pop=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/populacao_ibge.zip', compression='zip',sep=';').rename(columns={'ano':'ANO','cod_municipio':'IBGE'})
ibge=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/IBGECcodMunUF.csv', sep=';').rename(columns={'CodIBGE':'IBGE','SiglaUF':'SIGLA'})
est=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/IBGEEstadosUF.csv', sep=';').rename(columns={'NO_UF':'ESTADO','SG_UF':'SIGLA'})
estcomp = ibge.merge(est, left_on='SIGLA', right_on='SIGLA')
popibge = estcomp.merge(pop, left_on='IBGE', right_on='IBGE')
popibge['ANO'] = popibge['ANO'].astype(object)
popibge = popibge.drop(['IBGE', 'IdtCadastroSRFMunicipioTom', 'CodTOM','mes','SIGLA'], axis=1)
popmun = popibge.groupby(by=['ANO','ESTADO','NomeMunicipio']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','mean')).reset_index()
popest = popmun.groupby(by=['ANO','ESTADO']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','sum')).reset_index()

popgas = popest.merge(dfg, left_on=['ANO','ESTADO'], right_on=['ANO','ESTADO'])
maispopmaisgas = popgas['NU_POPULACAO_ESTIMADA'].corr(popgas['TOTAL_VENDAS'])

print('O quoeficiente de correlação entre quanto maior a população de um estado mais diesel será vendido é de ',maispopmaisgas, '. Seguramente, exite uma forte correlação entre estas variáveis.')

O quoeficiente de correlação entre quanto maior a população de um estado mais diesel será vendido é de  0.9328818394065902 . Seguramente, exite uma forte correlação entre estas variáveis.


QUESTÃO 8 - Qual foi o custo mensal de diesel S-10 no DF para o ano de 2020.Considere a média dos preços coletados e que o dataset de venda apresenta dados em metros cúbicos (e o de preço em R$/litro).


In [8]:
import pandas as pd

df1 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv', delimiter=';',decimal=',')
df2 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-02.csv', delimiter=';',decimal=',')



In [9]:
dfcomb = pd.concat([df1, df2]).rename(columns={'Estado - Sigla':'ESTADO', 'Valor de Venda':'PRECO','Data da Coleta':'DATA'})
dfcomb['DATA'] = pd.to_datetime(dfcomb['DATA'])
dfcomb = dfcomb.drop(['Regiao - Sigla', 'Municipio', 'Revenda','CNPJ da Revenda',	'Nome da Rua', 'Numero Rua',	'Complemento',	'Bairro',	'Cep',	'Valor de Compra',	'Bandeira'], axis=1)
dfcomf = dfcomb[dfcomb.Produto.eq('DIESEL S10')]
dfcomf = dfcomf[dfcomf.ESTADO.eq('DF')]

dfcommed = dfcomf.groupby(by=['ESTADO','Produto', 'DATA','Unidade de Medida']).agg(PRECO=('PRECO','mean')).reset_index()
dfmes = dfcommed.groupby(dfcommed.DATA.dt.month).mean()
print('O preço médio do OLEO DIESER S-10 em cada mês do ano de 2020 no DF foi de:')
print(dfmes)


O preço médio do OLEO DIESER S-10 em cada mês do ano de 2020 no DF foi de:
      PRECO
DATA       
1      3.81
2      3.83
3      3.79
4      3.64
5      3.46
6      3.45
7      3.51
8      3.72
9      3.56
10     3.76
11     3.59
12     3.87


QUESTÃO 9 -  Qual estado em 2020 gastou mais com diesel S-10 em relação a população? (um estado com 10 pessoas e que gasta 1000 reais com diesel em 2020 tem uma relação de R$ 100 reais gastos com diesel por pessoa).

In [ ]:
import pandas as pd

df1 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv', delimiter=';',decimal=',')
df2 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-02.csv', delimiter=';',decimal=',')

In [ ]:
import pandas as pd

df = pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/base-diesel-2013-2020.csv', sep=';', decimal=',')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df['TOTAL'] = df['TOTAL'].astype(float)
df['ANO'] = df['ANO'].astype(object)
dfc = df.drop(['UNIDADE', 'REGIÃO', 'Jan','Fev',	'Mar', 'Abr',	'Mai',	'Jun',	'Jul',	'Ago',	'Set',	'Out',	'Nov'	,'Dez'], axis=1)
dfg = dfc[dfc.ANO.eq(2020)]
dfg = dfg[dfg.COMBUSTÍVEL.eq('ÓLEO DIESEL S-10 ')]
dfg = dfg.groupby(by=['ANO','ESTADO','COMBUSTÍVEL']).agg(TOTAL_VENDAS=('TOTAL','mean')).reset_index()


pop=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/populacao_ibge.zip', compression='zip',sep=';').rename(columns={'ano':'ANO','cod_municipio':'IBGE'})
ibge=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/IBGECcodMunUF.csv', sep=';').rename(columns={'CodIBGE':'IBGE','SiglaUF':'SIGLA'})
est=pd.read_csv('https://github.com/leodjn/Pos_IESB/raw/main/IBGEEstadosUF.csv', sep=';').rename(columns={'NO_UF':'ESTADO','SG_UF':'SIGLA'})
estcomp = ibge.merge(est, left_on='SIGLA', right_on='SIGLA')
popibge = estcomp.merge(pop, left_on='IBGE', right_on='IBGE')
popibge['ANO'] = popibge['ANO'].astype(object)
popibge = popibge.drop(['IBGE', 'IdtCadastroSRFMunicipioTom', 'CodTOM','mes'], axis=1)
popmun = popibge.groupby(by=['ANO','ESTADO','NomeMunicipio','SIGLA']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','mean')).reset_index()
popest = popmun.groupby(by=['ANO','ESTADO','SIGLA']).agg(NU_POPULACAO_ESTIMADA=('NU_POPULACAO_ESTIMADA','sum')).reset_index()
popest = popest[popest.ANO.eq(2020)]


dfcomb = pd.concat([df1, df2]).rename(columns={'Estado - Sigla':'ESTADO', 'Valor de Venda':'PRECO','Data da Coleta':'DATA'})
dfcomb['DATA'] = pd.to_datetime(dfcomb['DATA'])
dfcomb = dfcomb.drop(['Regiao - Sigla', 'Municipio', 'Revenda','CNPJ da Revenda',	'Nome da Rua', 'Numero Rua',	'Complemento',	'Bairro',	'Cep',	'Valor de Compra',	'Bandeira'], axis=1)
dfcomb['ANO'] = dfcomb.DATA.dt.year
dfcommed = dfcomb.groupby(by=['ESTADO', 'ANO',]).agg(PRECO=('PRECO','mean')).reset_index()

popgas = popest.merge(dfg, left_on=['ANO','ESTADO'], right_on=['ANO','ESTADO'])
gascusto = popgas.merge(dfcommed, left_on=['SIGLA'], right_on=['ESTADO']) 
gascusto = gascusto.drop(['ANO_x', 'SIGLA', 'ESTADO_y'], axis=1).rename(columns={'ESTADO_x':'ESTADO', 'NU_POPULACAO_ESTIMADA':'POPULAÇÃO','TOTAL_VENDAS':'VENDAS', 'ANO_y':'ANO' })

gascusto['GASTO'] = gascusto['VENDAS']*gascusto['PRECO']
gascusto['QUOCIENTE'] = gascusto['GASTO']/gascusto['POPULAÇÃO']


gascusto.head()

maior = gascusto.sort_values(['QUOCIENTE'],ascending=[False]).head(1)
print('O estado que em 2020 gastou mais com diesel S-10 em relação a população foi:')
print(maior)